In [1]:
import pandas as pd
import os
import gc
import pickle

from time import sleep

import gspread_dataframe as gs_df

os.chdir("c:/Users/rsilva/Google Drive/projects/programming/automation")

# INITIALIZE DRIVER
from selenium import webdriver
from sapiens import LoginPage, KEYS, Page_sapiens
import functions


% reload_ext autoreload
% autoreload 2

row = 2

### Loga Driver no Sapiens

In [106]:
page = functions.init_browser(webdriver.Firefox(), 'rsilva', 'Savorthemom3nts')
driver = LoginPage(page.driver).login('33831640807', 'Fer465023@ur_')    

### Criar o Data Frame para exportar os dados

In [180]:
database = pd.read_csv("registros_cadin.csv")

In [181]:
database = database.dropna(subset=["CPF"])

In [182]:
registros = {row[1]["CPF"]:{k:v for k,v in row[1].items()} for row in df.iterrows()}

In [183]:
len(registros)        

6217

In [108]:
#sapiens = Page_sapiens(driver, {})

sapiens.reset_driver(driver)

### Leitura dos Dados Brutos na Planilha

In [109]:
gc = functions.authenticate_google("files/anatel.json")
sht = gc.open(title='RC')

In [110]:
registros = sht.worksheet('Registros')
registros_rf = sht.worksheet('Registros_RF')
df = pd.DataFrame(columns=KEYS)

In [111]:
lista_cpf = registros.col_values(1)[1:]

df.head()

,CPF,Nome,Mãe,Data de Nascimento,Sexo,Ano do Óbito,Nacionalidade,Título de Eleitor,Situação Cadastral,Fone,Logradouro,Número,Complemento,Bairro,Cidade,Cep


In [121]:
df.shape

(1778, 16)

In [118]:
counter = 0

for cpf in lista_cpf:
    
    cpf = str(cpf)    
        
    if len(cpf) in (12, 13, 14):
        
        while len(cpf) < 14:
            
            cpf = '0' + str(cpf)
            
    elif len(cpf) <= 11:
    
        while len(cpf) < 11:

            cpf = '0' + str(cpf)
            
    else:
        
        raise ValueError("CPF/CNPJ inválido {}".format(cpf))

    registro = sapiens.get_registro(cpf)    
   
    if not registro:
    
        sapiens.pesquisa_dados(cpf)
            
        registro = sapiens.get_registro(cpf)
        
        counter += 1
        
   
    if not (df["CPF"] == registro.get(cpf)).any():
        
        df = df.append(pd.Series(registro), ignore_index=True)
        
    if counter > 100:
        
        #df.to_csv("registros_cadin.csv", index=False)
        
        page = functions.init_browser(webdriver.Firefox(), 'rsilva', 'Savorthemom3nts')
        
        driver = LoginPage(page.driver).login('33831640807', 'Fer465023@ur_')        
               
        sapiens.reset_driver(driver)
        
        sleep(10)        
                
        counter = 0  

In [122]:
df = df.drop_duplicates()
df = df.fillna("")

In [123]:
df["Endereço_Completo"] = df.apply(lambda row: " ".join([str(row["Logradouro"]), 
                                                         str(row["Número"]), 
                                                         str(row["Complemento"]),
                                                         str(row["Bairro"]), "\n", "CEP:", 
                                                         str(row["Cep"]), 
                                                         str(row["Cidade"])]
                                                       ), axis=1
                                  )

In [125]:
df = df.sort_values(by="Nome")

In [65]:
#writer = pd.ExcelWriter("files/planilha_obito.xlsx")
#df.to_excel(writer, "consulta_RF", index=False, encoding='utf-8')
#writer.save()

In [129]:
#gs_df.set_with_dataframe(registros_rf, df, row=1, include_column_header=True)
gs_df.set_with_dataframe(registros, pd.DataFrame(df["Nome"]), row=1, col=9, include_column_header=False)
gs_df.set_with_dataframe(registros, pd.DataFrame(df["Endereço_Completo"]), row=1, col=12, include_column_header=False)



In [94]:
len(sapiens.registros)

896

In [127]:
df.head()

,CPF,Nome,Mãe,Data de Nascimento,Sexo,Ano do Óbito,Nacionalidade,Título de Eleitor,Situação Cadastral,Fone,Logradouro,Número,Complemento,Bairro,Cidade,Cep,Endereço_Completo
822,,,,,,,,,ATIVA,,,,,,,,\n CEP:
0,858.388.968-68,ABELARDO DOS SANTOS,JULIETA PEREIRA DOS SANTOS,29-07-1950,MASCULINO,,BRASILEIRO,0071385460116,REGULAR,32251419,OUTROS GENESIO MARIA,41,,BOA ESPERANCA,SOROCABA/SP,18103-185,OUTROS GENESIO MARIA 41 BOA ESPERANCA \n CEP:...
1,052.347.758-95,ABILIO GONCALVES PRETO,ANA MARIA PRETO,08-05-1964,MASCULINO,,BRASILEIRO,0019319610141,REGULAR,35248858,RUA URUGUAI,62,,VILA JUCA PEDRO,CATANDUVA/SP,15800-420,RUA URUGUAI 62 VILA JUCA PEDRO \n CEP: 15800-...
2,172.035.218-69,ACACIO SANTIAGO GALA,,15-03-1972,MASCULINO,,BRASILEIRO,0257145460108,REGULAR,32677461,RUA PEDREIRA,292,CASA,JD N CAMPOS ELISEOS,CAMPINAS/SP,13050-544,RUA PEDREIRA 292 CASA JD N CAMPOS ELISEOS \n C...
3,081.843.968-80,ADALBERTO CARDOSO,PEDRINA BORELLI CARDOSO,17-02-1966,MASCULINO,,BRASILEIRO,0104874860141,REGULAR,32211243,RUA ORSINI BERTANHA,36,,JARDIM BERTANHA,SOROCABA/SP,18052-170,RUA ORSINI BERTANHA 36 JARDIM BERTANHA \n CEP...
